# Aula 1 - Buscando a similaridade

In [1]:
url = 'https://raw.githubusercontent.com/Mirlaa/NLP-trabalhando-similaridade-sentencas/refs/heads/main/reviews_zoop.csv'

In [2]:
import pandas as pd

dados = pd.read_csv(url)

dados

,nota_review,review
0,5,Recebi bem antes do prazo estipulado.
1,5,Parabéns lojas Zoop adorei comprar pela Intern...
2,4,aparelho eficiente. no site a marca do aparelh...
3,4,"Mas um pouco ,travando...pelo valor ta Boa.\r\n"
4,5,"Vendedor confiável, produto ok e entrega antes..."
...,...,...
39457,5,Entregou dentro do prazo. O produto chegou em ...
39458,3,"O produto não foi enviado com NF, não existe v..."
39459,5,"Excelente mochila, entrega super rápida. Super..."
39460,1,Solicitei a compra de uma capa de retrovisor c...


In [3]:
dados['review'][1]

'Parabéns lojas Zoop adorei comprar pela Internet seguro e prático Parabéns a todos feliz Páscoa'

In [4]:
dados['review'][8]

'A compra foi realizada facilmente.\r\nA entrega foi efetuada muito antes do prazo dado.\r\nO produto já começou a ser usado e até o presente,\r\nsem problemas.'

In [5]:
dados['review'][11]

'Sempre compro pela Internet e a entrega ocorre antes do prazo combinado, que acredito ser o prazo máximo. No  o prazo máximo já se esgotou e ainda não recebi o produto.'

In [6]:
dados['review'][18]

'otimo vendedor chegou ate antes do prazo , adorei o produto'

## Similaridade entre as palavras

In [7]:
from nltk.tokenize import word_tokenize
from functools import partial

word_tokenize_pt = partial(word_tokenize, language='portuguese')

In [8]:
import nltk

nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [9]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 67.4 MB/s eta 0:00:00


In [10]:
from gensim.models import Word2Vec

modelo = Word2Vec(sentences=dados['review'].apply(word_tokenize_pt),
                  vector_size=100, min_count=1, window=5,
                  workers=1, seed=45)

In [11]:
modelo.wv.most_similar('entrega')

[('Entrega', 0.7579216361045837),
 ('entregue', 0.7184001207351685),
 ('compra', 0.7105124592781067),
 ('chegou', 0.6817407608032227),
 ('entregou', 0.6612473130226135),
 ('preparar', 0.6325778365135193),
 ('envio', 0.6313653588294983),
 ('sorriso', 0.6210829615592957),
 ('Chegou', 0.6198225021362305),
 ('mercadoria', 0.6146730780601501)]

In [12]:
modelo.wv.most_similar('vendedor')

[('atendimento', 0.8572612404823303),
 ('serviço', 0.8074656128883362),
 ('fornecedor', 0.7500720024108887),
 ('veiculo', 0.7239760756492615),
 ('produto', 0.7152891755104065),
 ('processo', 0.7085978388786316),
 ('parceiro', 0.7057496905326843),
 ('cumpriu', 0.6909526586532593),
 ('trabalho', 0.6848700642585754),
 ('cumpriram', 0.6713030934333801)]

In [13]:
modelo.wv.most_similar('produto')

[('pedido', 0.749972939491272),
 ('relógio', 0.7460570335388184),
 ('vendedor', 0.7152891755104065),
 ('endereço', 0.7050330638885498),
 ('Produto', 0.6978251934051514),
 ('mesmo', 0.6896668076515198),
 ('tempo', 0.6797294616699219),
 ('Mas', 0.6640905737876892),
 ('mercadoria', 0.6585670113563538),
 ('condizia', 0.6553379893302917)]

## Similaridade entre sentenças

In [14]:
dados['review_token'] = dados['review'].apply(word_tokenize_pt)

In [15]:
from gensim.models.doc2vec import TaggedDocument

dados_tag = [TaggedDocument(words=linha['review_token'], tags=[str(i)]) for i, linha in dados.iterrows()]

In [16]:
dados_tag

[TaggedDocument(words=['Recebi', 'bem', 'antes', 'do', 'prazo', 'estipulado', '.'], tags=['0']),
 TaggedDocument(words=['Parabéns', 'lojas', 'Zoop', 'adorei', 'comprar', 'pela', 'Internet', 'seguro', 'e', 'prático', 'Parabéns', 'a', 'todos', 'feliz', 'Páscoa'], tags=['1']),
 TaggedDocument(words=['aparelho', 'eficiente', '.', 'no', 'site', 'a', 'marca', 'do', 'aparelho', 'esta', 'impresso', 'como', '3desinfector', 'e', 'ao', 'chegar', 'esta', 'com', 'outro', 'nome', '...', 'atualizar', 'com', 'a', 'marca', 'correta', 'uma', 'vez', 'que', 'é', 'o', 'mesmo', 'aparelho'], tags=['2']),
 TaggedDocument(words=['Mas', 'um', 'pouco', ',', 'travando', '...', 'pelo', 'valor', 'ta', 'Boa', '.'], tags=['3']),
 TaggedDocument(words=['Vendedor', 'confiável', ',', 'produto', 'ok', 'e', 'entrega', 'antes', 'do', 'prazo', '.'], tags=['4']),
 TaggedDocument(words=['GOSTARIA', 'DE', 'SABER', 'O', 'QUE', 'HOUVE', ',', 'SEMPRE', 'RECEBI', 'E', 'ESSA', 'COMPRA', 'AGORA', 'ME', 'DECPCIONOU'], tags=['5']),
 T

In [17]:
from gensim.models import Doc2Vec

modelo = Doc2Vec(documents=dados_tag, vector_size=100, min_count=2, window=2, workers=1, seed=45, epochs=20)

In [18]:
vetor_inferido = modelo.infer_vector(['entrega'])

In [19]:
vetor_inferido

array([-0.10694628, -0.06644846,  0.06811451, -0.04838789,  0.08350792,
        0.06128964, -0.03182549, -0.0230466 ,  0.01414252, -0.09645917,
       -0.10675522,  0.02424835, -0.08946601,  0.0340785 , -0.0802439 ,
       -0.07200859, -0.04866432,  0.0197278 , -0.03550905,  0.01343846,
        0.03246605, -0.0305418 ,  0.08440375, -0.02469276, -0.02371004,
       -0.00610899, -0.09624264, -0.08051746, -0.03674939, -0.04739986,
        0.06296933, -0.02428986,  0.01972906,  0.11618809,  0.04012979,
       -0.0399715 , -0.10869766,  0.14692627, -0.10434925,  0.04509829,
        0.01664585,  0.06027739,  0.00502434, -0.0070791 , -0.00824591,
       -0.10093506,  0.03459381,  0.00304967, -0.04884594, -0.00293419,
        0.00460248, -0.01840098,  0.03175429, -0.02502802,  0.10147472,
       -0.11712827, -0.02444071, -0.02566587, -0.04801856, -0.07013851,
        0.02432177, -0.03342374,  0.02085617, -0.06739717,  0.02815027,
       -0.01662399,  0.05818867,  0.00249393,  0.02824287,  0.03

In [20]:
frases_similares = modelo.dv.most_similar([vetor_inferido], topn=5)
frases_similares

[('24884', 0.8528653979301453),
 ('24666', 0.8475314378738403),
 ('9304', 0.8440483808517456),
 ('33496', 0.8341901898384094),
 ('30445', 0.823677122592926)]

In [21]:
for idx, similaridade in frases_similares:
    print(f"Review: {dados.iloc[int(idx)]['review']} - Similaridade: {similaridade:.4f}")

Review: entrega antes do prazo - Similaridade: 0.8529
Review: entrega antes do prazo - Similaridade: 0.8475
Review: entrega no prazo. - Similaridade: 0.8440
Review: Ainda estou esperando a entrega!  - Similaridade: 0.8342
Review: Comprem nas Zoop entrega certaaaa - Similaridade: 0.8237


# Aula 2 - Ajustando os textos de review

In [24]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [25]:
import re
from nltk.corpus import stopwords

def tratamento_inicial(texto):
  texto=re.sub(r'\W', ' ', texto.lower())

  tokens = word_tokenize_pt(texto)
  stop_words = set(stopwords.words('portuguese'))
  stop_words.discard('não')

  return ' '.join([w for w in tokens if w not in stop_words])



In [26]:
dados['tratamento_1'] = dados['review'].apply(tratamento_inicial)

In [27]:
dados['tratamento_1']

,tratamento_1
0,recebi bem antes prazo estipulado
1,parabéns lojas zoop adorei comprar internet se...
2,aparelho eficiente site marca aparelho impress...
3,pouco travando valor ta boa
4,vendedor confiável produto ok entrega antes prazo
...,...
39457,entregou dentro prazo produto chegou condições...
39458,produto não enviado nf não existe venda nf cer...
39459,excelente mochila entrega super rápida super r...
39460,solicitei compra capa retrovisor celta prisma ...


In [29]:
dados['review'][11]

'Sempre compro pela Internet e a entrega ocorre antes do prazo combinado, que acredito ser o prazo máximo. No  o prazo máximo já se esgotou e ainda não recebi o produto.'

In [30]:
dados['tratamento_1'][11]

'sempre compro internet entrega ocorre antes prazo combinado acredito prazo máximo prazo máximo esgotou ainda não recebi produto'

## Uniformizando o texto

In [31]:
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 5.1 MB/s eta 0:00:00


In [32]:
import unidecode

sem_acentos = [unidecode.unidecode(texto) for texto in dados['tratamento_1']]

In [33]:
dados['tratamento_2'] = sem_acentos

In [34]:
dados['tratamento_2']

,tratamento_2
0,recebi bem antes prazo estipulado
1,parabens lojas zoop adorei comprar internet se...
2,aparelho eficiente site marca aparelho impress...
3,pouco travando valor ta boa
4,vendedor confiavel produto ok entrega antes prazo
...,...
39457,entregou dentro prazo produto chegou condicoes...
39458,produto nao enviado nf nao existe venda nf cer...
39459,excelente mochila entrega super rapida super r...
39460,solicitei compra capa retrovisor celta prisma ...


In [35]:
dados['review_retratamento'] = dados['tratamento_2'].apply(word_tokenize_pt)
dados_tag = [TaggedDocument(words=linha['review_token'], tags=[str(i)]) for i, linha in dados.iterrows()]
modelo = Doc2Vec(documents=dados_tag, vector_size=100, min_count=2, window=2, workers=1, seed=45, epochs=20)
frases_similares = modelo.dv.most_similar([modelo.infer_vector(['entrega'])], topn=5)
for idx, similaridade in frases_similares:
    print(f"Review: {dados.iloc[int(idx)]['review']} - Similaridade: {similaridade:.4f}")

Review: entrega antes do prazo - Similaridade: 0.8529
Review: entrega antes do prazo - Similaridade: 0.8475
Review: entrega no prazo. - Similaridade: 0.8440
Review: Ainda estou esperando a entrega!  - Similaridade: 0.8342
Review: Comprem nas Zoop entrega certaaaa - Similaridade: 0.8237


## Correções nos dados

In [36]:
df = dados.drop_duplicates(subset='tratamento_2')
df

,nota_review,review,review_token,tratamento_1,tratamento_2,review_retratamento
0,5,Recebi bem antes do prazo estipulado.,"[Recebi, bem, antes, do, prazo, estipulado, .]",recebi bem antes prazo estipulado,recebi bem antes prazo estipulado,"[recebi, bem, antes, prazo, estipulado]"
1,5,Parabéns lojas Zoop adorei comprar pela Intern...,"[Parabéns, lojas, Zoop, adorei, comprar, pela,...",parabéns lojas zoop adorei comprar internet se...,parabens lojas zoop adorei comprar internet se...,"[parabens, lojas, zoop, adorei, comprar, inter..."
2,4,aparelho eficiente. no site a marca do aparelh...,"[aparelho, eficiente, ., no, site, a, marca, d...",aparelho eficiente site marca aparelho impress...,aparelho eficiente site marca aparelho impress...,"[aparelho, eficiente, site, marca, aparelho, i..."
3,4,"Mas um pouco ,travando...pelo valor ta Boa.\r\n","[Mas, um, pouco, ,, travando, ..., pelo, valor...",pouco travando valor ta boa,pouco travando valor ta boa,"[pouco, travando, valor, ta, boa]"
4,5,"Vendedor confiável, produto ok e entrega antes...","[Vendedor, confiável, ,, produto, ok, e, entre...",vendedor confiável produto ok entrega antes prazo,vendedor confiavel produto ok entrega antes prazo,"[vendedor, confiavel, produto, ok, entrega, an..."
...,...,...,...,...,...,...
39457,5,Entregou dentro do prazo. O produto chegou em ...,"[Entregou, dentro, do, prazo, ., O, produto, c...",entregou dentro prazo produto chegou condições...,entregou dentro prazo produto chegou condicoes...,"[entregou, dentro, prazo, produto, chegou, con..."
39458,3,"O produto não foi enviado com NF, não existe v...","[O, produto, não, foi, enviado, com, NF, ,, nã...",produto não enviado nf não existe venda nf cer...,produto nao enviado nf nao existe venda nf cer...,"[produto, nao, enviado, nf, nao, existe, venda..."
39459,5,"Excelente mochila, entrega super rápida. Super...","[Excelente, mochila, ,, entrega, super, rápida...",excelente mochila entrega super rápida super r...,excelente mochila entrega super rapida super r...,"[excelente, mochila, entrega, super, rapida, s..."
39460,1,Solicitei a compra de uma capa de retrovisor c...,"[Solicitei, a, compra, de, uma, capa, de, retr...",solicitei compra capa retrovisor celta prisma ...,solicitei compra capa retrovisor celta prisma ...,"[solicitei, compra, capa, retrovisor, celta, p..."


In [37]:
df['review'][120]

'A'

In [38]:
df['tratamento_2'][120]

''

In [39]:
df = df[df['tratamento_2'] != '']

In [40]:
df['tratamento_2'][3319]

'simplismeente ameiii'

In [41]:
df['tratamento_2'][8354]

'recebi rapido antes prazo otimo qualidade adoreii'

In [42]:
df['tratamento_2'][12843]

'produto chegou direitinho antes muuuito antes prazo superou expectativas'

In [44]:
def normalizar_repeticoes(texto):
  return re.sub(r'(?!rr|ss)(.)\1+', r'\1', texto)

df['tratamento_3'] = df['tratamento_2'].apply(normalizar_repeticoes)

/tmp/ipython-input-1120237711.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tratamento_3'] = df['tratamento_2'].apply(normalizar_repeticoes)


In [45]:
df['tratamento_3'][3319]

'simplismente amei'

In [46]:
df['tratamento_3'][8354]

'recebi rapido antes prazo otimo qualidade adorei'

In [47]:
df.reset_index(drop=True, inplace=True)

## Lematização

In [48]:
!pip install stanza
!python -m stanza.download pt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 32.0 MB/s eta 0:00:00
/usr/bin/python3: No module named stanza.download


In [49]:
import stanza

stanza.download('pt')
nlp = stanza.Pipeline('pt', processors='tokenize,lemma', use_gpu=False, batch_size=64, n_pross=4)

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: pt (Portuguese) ...


INFO:stanza:Downloaded file to /root/stanza_resources/pt/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources
INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: pt (Portuguese):
| Processor | Package         |
-------------------------------
| tokenize  | bosque          |
| mwt       | bosque          |
| lemma     | bosque_nocharlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: lemma
INFO:stanza:Done loading processors!


In [50]:
def lematizar_texto(textos):
  texto_lematizados = []

  for texto in textos:
    doc_frase = nlp(texto)
    frase_lematizada = ' '.join([palavra.lemma for frase in doc_frase.sentences for palavra in frase.words])
    texto_lematizados.append(frase_lematizada)

  return texto_lematizados

In [51]:
textos = ['gostei muito experiencia comprar',
          'minha filha gostou produto',
          'compra foi facil compra rapida']
lematizar_texto(textos)

['gostar muito experiencia comprar',
 'meu filha gostar produto',
 'compra ser facil compra rapida']

In [52]:
df['tratamento_4'] = lematizar_texto(df['tratamento_3'])

/tmp/ipython-input-503408071.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tratamento_4'] = lematizar_texto(df['tratamento_3'])


In [53]:
df['tratamento_4']

,tratamento_4
0,recebi bem antes prazo estipulado
1,parabem loja zop adorei comprar internet segur...
2,aparelho eficiente site marca aparelho imprimi...
3,pouco travar valor to bom
4,vendedor confiavel produto ok entregar antes p...
...,...
31469,entregar dentro prazo produto chegar condicoe ...
31470,produto nao enviado em f nao existir venda em ...
31471,excelente mochila entregar super rapida super ...
31472,solicitei compra capa retrovisor celta prisma ...


# Aula 3 - Otimizando o Doc2vec

## Treinamento do Doc2vec

## Verificando a similardade - Doc2Vec

# Aula 4 - Considerando o significado das frases

In [ ]:
modulo_url = 'https://tfhub.dev/google/universal-sentence-encoder/4'

## Avaliando as sentenças similares

# Aula 5 - Transformando a similaridade em aplicação

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')

modulo_url = 'https://tfhub.dev/google/universal-sentence-encoder/4'


